# System Checker - AWS 1

Der ```System Checker``` überprüft welche Versionen der für die Lehrveranstaltung "Angewandte Systemwissenschaften 1" in der "Gruppe Plakolb" **relevanten Software** installiert sind.

Python-Pakete und Anaconda-Pakete werden dabei **automatisch installiert**, sollten sie noch nicht vorhanden sein.

Zum Schluss erstellt der ```System Checker``` einen Report über etwaige Fehler bzw die Versionsnummern. Dieser kann über Moodle an die LV-Leitung übermittelt werden.

### Anaconda

Zur Installation von Anaconda bitte diesem Link folgen. ```#TODO```

### Editor

Grundsätzlich ist es in der LV **egal** welcher Text-Editor/welche IDE individuell verwendet wird. Frühere Teilnehmer waren neben dem rudimentären, von [Jupyter](https://jupyter.org/) zur Verfügung gestellten Editor zufrieden mit:

* [Visual Studio Code](https://code.visualstudio.com/)
* [PyCharm](https://www.jetbrains.com/pycharm/)
* [Spyder](https://www.spyder-ide.org/)

Wichtig ist, dass man sich mit der Lösung wohlfühlt :)

### Sinn und Zweck

Der Grund für diesen Check ist, dass gleich zu Beginn der Lehrveranstaltung eine solide Ausgangsbasis auf allen verwendeten Geräten sichergestellt werden soll.

In [1]:
import os
import sys
import subprocess
import conda
import importlib
import socket
import getpass
import datetime
from functools import reduce

run = lambda *cmd: subprocess.run(cmd, capture_output=True)

### Helper Functions

In [2]:
def check_version(name, failed=False):
    """Check the version of a command line tool ``name``.
    
    Parameters
    ----------
    name : str
        The name of the cli tool.
        
    Returns
    -------
    version : str
        ``stdout`` output of a ``name --version`` call.
    """
    version = run(name,'--version')
    if version.stderr:
        install = run('conda','install','-y',name)
        return install
    else:
        return version

def check_import(name, failed=False):
    """Check the version of a Python library ``name``.
    
    Parameters
    ----------
    name : str
        The name of the library.
        
    Returns
    -------
    version : str
        The output of ``name.__version__``.
    """
    try:
        m = importlib.import_module(name)
        res = m.__version__
    except ImportError as e:
        if not failed:
            run('pip','install',name)
            return check_import(name, failed=True)
        else:
            res = repr(e)
    return res

## Check conda path

The installation path of anaconda cannot contain any space characters (```' '```).

In [3]:
conda_path_ok = conda.CONDA_PACKAGE_ROOT.find(' ') < 0

In [4]:
conda_version = run('conda','--version')

In [5]:
# check version
if int(''.join(conda_version.stdout.decode('utf-8').strip()[-5:].split('.'))) < 400:
    conda_update = run('conda','update','-y','conda')
else:
    conda_update = None

In [6]:
conda = run('conda','info')

In [7]:
conda_env = run('conda','env','list')

In [8]:
# conda_update_all = run('conda','update','-y','--all')

## Check conda installs

In [9]:
python = check_version('python')
pip = check_version('pip')
git = check_version('git')
jupyter = check_version('jupyter')

In [10]:
%%capture vscode
# VS Code triggers FileNotFoundError when "run('code')"  on Windows
!code --version

## Check python packages

In [11]:
matplotlib = check_import('matplotlib')
numpy = check_import('numpy')
pandas = check_import('pandas')
pip_py = check_import('pip')
pytest = check_import('pytest')
hypothesis = check_import('hypothesis')

In [12]:
%%capture matplotlib_backend
%matplotlib

## Print Report

In [13]:
def write_report(*contents):
    """Output a report file with ``contents``.
    
    Parameters
    ----------
    *contents : Variable tuple of strings
        The lines that should be printed to the file.
    """
    with open('syscheck-report-{}-{}-{}.txt'.format(
                socket.gethostname(),
                getpass.getuser(),
                datetime.datetime.now().date().isoformat()
                )
              , 'w') as f:
        
        for line in contents:
            if line:
                f.write(line + '\n')
        else:
            f.write('\n'+hex(hash(reduce(lambda a,b: a+b, contents))))

In [14]:
write_report(
    'Report',
    '------',
    '\nSystem',
    '------\n',
    '- Platform: {}'.format(sys.platform),
    '- Conda path clean: {}'.format(conda_path_ok),
    '- Conda cli:',
    conda.stdout.decode('utf-8'),
    conda.stderr.decode('utf-8'),
    conda_env.stdout.decode('utf-8'),
    conda_env.stderr.decode('utf-8'),
#     conda_update_all.stdout.decode('utf-8'),
#     conda_update_all.stderr.decode('utf-8'),
    '\n- Python cli:',
    python.stdout.decode('utf-8'),
    python.stderr.decode('utf-8'),
    '\n- Pip cli:',
    pip.stdout.decode('utf-8'),
    pip.stderr.decode('utf-8'),
    '\n- Git cli:',
    git.stdout.decode('utf-8'),
    git.stderr.decode('utf-8'),
    '\n-Jupyter cli:',
    jupyter.stdout.decode('utf-8'),
    jupyter.stderr.decode('utf-8'),
    '\n- Visual Studio Code:',
    vscode.stdout,
    vscode.stderr,
    '\nPython modules',
    '--------------\n',
    '- matplotlib: ' + matplotlib,
    '- Backend: ' + matplotlib_backend.stdout,
    matplotlib_backend.stderr,
    '- mumpy: ' + numpy,
    '- pandas: ' + pandas,
    '- pip: ' + pip_py,
    '- pytest: ' + pytest,
    '- hypothesis: ' + hypothesis
    )